https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder

In [20]:
from absl import logging
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import pandas
import scipy
import math
import csv

BASE_PATH = "../../../../local_data/practice/tensorflow_hub/"
DATA_PATH = "../../../../local_data/tfds/"
OUTPUT_PATH = BASE_PATH+"use_tutorial_stsbenchmark/"
os.system("mkdir -p " + OUTPUT_PATH)

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [21]:
sts_dataset = tf.keras.utils.get_file(
    fname="Stsbenchmark.tar.gz",
    origin="http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz",
    extract=True)
sts_dev = pandas.read_table(
    os.path.join(os.path.dirname(sts_dataset), "stsbenchmark", "sts-dev.csv"),
    skip_blank_lines=True,
    usecols=[4, 5, 6],
    names=["sim", "sent_1", "sent_2"])
sts_test = pandas.read_table(
    os.path.join(
        os.path.dirname(sts_dataset), "stsbenchmark", "sts-test.csv"),
    quoting=csv.QUOTE_NONE,
    skip_blank_lines=True,
    usecols=[4, 5, 6],
    names=["sim", "sent_1", "sent_2"])
# cleanup some NaN values in sts_dev
sts_dev = sts_dev[[isinstance(s, str) for s in sts_dev['sent_2']]]

In [22]:
pd.set_option('expand_frame_repr', False)
# print(sts_dev)

In [23]:
# print(embed(["A man with a hard hat is dancing."]))

In [24]:
pd.set_option('expand_frame_repr', False)
# print(sts_test)

In [25]:
def run_sts_benchmark_short(batch):
  sts_encode1 = tf.nn.l2_normalize(embed(tf.constant(batch['sent_1'].tolist())), axis=1)
  sts_encode2 = tf.nn.l2_normalize(embed(tf.constant(batch['sent_2'].tolist())), axis=1)
  cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
  clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
  scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi
  """Returns the similarity scores"""
  return scores

sts_data = sts_dev

dev_scores = sts_data['sim'].tolist()
scores = []
for batch in np.array_split(sts_data, 10):
  scores.extend(run_sts_benchmark_short(batch))

pearson_correlation = scipy.stats.pearsonr(scores, dev_scores)
print('Pearson correlation coefficient = {0}\np-value = {1}'.format(
    pearson_correlation[0], pearson_correlation[1]))

/media/data/miniconda3/envs/jh_class/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Pearson correlation coefficient = 0.8036394605028534
p-value = 0.0


In [26]:
def run_sts_benchmark_long(batch):
  b1=tf.constant(batch['sent_1'].tolist())
  b2=tf.constant(batch['sent_2'].tolist())
  # print("b1=",b1)
  # print("b2=",b2)
  e1=embed(b1)
  e2=embed(b2)
  # print("e1=",e1)
  sts_encode1 = tf.nn.l2_normalize(e1, axis=1)
  sts_encode2 = tf.nn.l2_normalize(e2, axis=1)
  # print("sts_encode1=",sts_encode1)
  # print("sts_encode2=",sts_encode2)
  mult=tf.multiply(sts_encode1, sts_encode2)
  # print("mult=",mult)
  # print("sts_encode1.shape=",sts_encode1.shape,"sts_encode2.shape=",sts_encode2.shape,"mult.shape=",mult.shape)
  cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
  # print(cosine_similarities)
  clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
  scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi
  """Returns the similarity scores"""
  return scores

sts_data = sts_dev

dev_scores = sts_data['sim'].tolist()
print("len(dev_scores)=",len(dev_scores))
print("dev_scores=",dev_scores)
scores = []
for batch in np.array_split(sts_data, 10):
  # print(batch)
  scores.extend(run_sts_benchmark_long(batch))

df_dev_scores=pd.DataFrame(dev_scores)
calc_scores=tf.convert_to_tensor(scores)
df_calc_scores=pd.DataFrame(calc_scores)
compare = pd.concat([df_dev_scores, df_calc_scores], axis=1)

# print(df_dev_scores)
# print(calc_scores)
# print(df_calc_scores)
print(compare)
compare.to_csv(OUTPUT_PATH + "pred_test_load.csv", index=False)   

len(dev_scores)= 1468
dev_scores= [5.0, 4.75, 5.0, 2.4, 2.75, 2.615, 5.0, 2.333, 3.75, 5.0, 3.2, 1.583, 5.0, 5.0, 4.909, 0.8, 2.4, 5.0, 4.0, 0.636, 3.0, 1.714, 3.2, 2.167, 1.0, 1.917, 4.25, 3.0, 1.0, 0.6, 2.6, 5.0, 4.6, 5.0, 4.8, 3.8, 5.0, 5.0, 4.2, 1.4, 3.6, 2.8, 1.6, 3.0, 1.4, 0.25, 0.25, 0.0, 4.0, 4.5, 0.5, 3.8, 4.8, 5.0, 0.25, 1.2, 0.6, 0.8, 3.8, 0.0, 3.5, 4.5, 2.8, 3.8, 3.8, 0.0, 4.0, 4.25, 2.812, 4.25, 3.0, 1.0, 3.75, 0.0, 0.4, 4.0, 2.8, 3.75, 1.154, 2.75, 2.8, 0.0, 0.0, 3.4, 5.0, 0.8, 2.25, 2.75, 4.5, 2.6, 3.8, 2.8, 1.6, 0.0, 1.333, 2.6, 0.6, 0.0, 0.0, 3.0, 0.0, 0.2, 3.2, 2.0, 0.0, 3.0, 3.091, 2.75, 1.2, 0.5, 0.25, 0.0, 0.0, 0.5, 0.0, 0.6, 4.0, 3.4, 0.0, 0.0, 1.0, 0.0, 2.4, 0.4, 1.8, 2.5, 0.083, 0.0, 1.0, 4.5, 0.0, 3.6, 5.0, 5.0, 5.0, 5.0, 3.8, 5.0, 2.5, 3.824, 1.6, 5.0, 5.0, 2.375, 5.0, 1.25, 1.25, 3.0, 2.167, 1.4, 4.0, 3.6, 4.857, 0.4, 0.2, 1.3, 1.2, 1.2, 4.4, 1.8, 1.4, 0.4, 4.4, 3.25, 0.778, 2.25, 2.75, 2.0, 0.833, 3.75, 1.8, 4.333, 4.214, 0.0, 3.4, 4.2, 0.8, 4.2, 2.75, 2.2, 

In [27]:
pearson_correlation = scipy.stats.pearsonr(scores, dev_scores)
print('Pearson correlation coefficient = {0}\np-value = {1}'.format(
    pearson_correlation[0], pearson_correlation[1]))

Pearson correlation coefficient = 0.8036396865485238
p-value = 0.0
